In [ ]:
#Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        pt = torch.exp(-bce_loss)# 
        focal = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal.mean()

# Model: ConvNeXt Base
model = create_model("convnext_base", pretrained=True, num_classes=1).to(DEVICE)
criterion = FocalLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Mixed Precision
scaler = torch.cuda.amp.GradScaler()

# Train function
def train():
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images, labels = images.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        scheduler.step()
        print(f"Epoch {epoch+1}: Train Loss = {total_loss/len(train_loader):.4f}")
        validate()

def validate():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
            outputs = torch.sigmoid(model(images))
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

train()
torch.save(model.state_dict(), "convnext_soil_classifier.pth")
print("Model saved to convnext_soil_classifier.pth")


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

/tmp/ipykernel_35/981703507.py:136: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1:   0%|          | 0/62 [00:00<?, ?it/s]/tmp/ipykernel_35/981703507.py:146: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1: 100%|██████████| 62/62 [00:22<00:00,  2.80it/s]

Epoch 1: Train Loss = 0.0057


Validation Accuracy: 99.80%


Epoch 2: 100%|██████████| 62/62 [00:22<00:00,  2.71it/s]

...

Epoch 9: Train Loss = 0.0004


Validation Accuracy: 100.00%


Epoch 10: 100%|██████████| 62/62 [00:22<00:00,  2.77it/s]

Epoch 10: Train Loss = 0.0001


Validation Accuracy: 100.00%
Model saved to convnext_soil_classifier.pth
